

Starter scripts in https://github.com/Gilles86/riskeye/blob/main/riskeye/eyetracking/

- Currently only did saccades, now also eyepos 

- `prep_eyetrack_data.py` 
    - get_experimental_messages --> sub-XX_messages.tsv
    - get_saccades ---> sub-XX_saccades.tsv

- subject.get_trialwise_saccade_info(source=source....

    def get_trialwise_saccade_info(self, source='saccades', summarize_trials=True):
        elif source == 'eyepos':
            eyepos = self.get_eyeposition(trialwise=True)
    
    def get_eyeposition(self, trialwise=True):
            fn = op.join(self.bids_folder, 'derivatives', 'pupil', f'sub-{self.subject_id}', 'func', f'sub-{self.subject_id}_run-{run}_gaze.tsv.gz')

--> where and how this '..._gaze.tsv.gz' created ?


In [1]:
import pandas as pd
import numpy as np
import os
import os.path as op


bids_folder = '/Users/mrenke/data/ds-dnumrisk'


In [2]:
from numrisk.behavior_risk.utils_eyetrack import Subject
subject_id = '01'

subject = Subject(subject_id, bids_folder)

In [3]:
subject.get_eyeposition()

L_gaze_X  L_gaze_Y fixation_target
subject trial time                                                      
01      1     0 days 02:29:11.122000     789.6     520.9     center_left
              0 days 02:29:11.124000     789.4     520.8     center_left
              0 days 02:29:11.126000     789.1     520.5     center_left
              0 days 02:29:11.128000     789.0     520.3     center_left
              0 days 02:29:11.130000     788.8     520.2     center_left
...                                        ...       ...             ...
        192   0 days 02:29:17.096000    1301.5     313.5    right_option
              0 days 02:29:17.098000    1301.1     313.7    right_option
              0 days 02:29:17.100000    1300.8     314.0    right_option
              0 days 02:29:17.102000    1300.7     314.3    right_option
              0 days 02:29:17.104000    1300.8     314.6    right_option

[175264 rows x 3 columns]

In [6]:
eyepos = subject.get_trialwise_saccade_info(source='eyepos',summarize_trials=False).sort_index()
eyepos.head()

n fixation_target  duration
subject trial                              
01      1       1     center_left     252.0
        1       2     left_option     646.0
        1       6    right_option     666.0
        1      10     left_option     740.0
        1      14    right_option     486.0

In [ ]:
# eyepos = self.get_eyeposition(trialwise=True)

trialwise=True

eyepos = []
fn = op.join(bids_folder, 'derivatives', 'pupil', f'sub-{subject_id}', 'func', f'sub-{subject_id}_gaze.tsv.gz')

def get_header_length_csv(fn):
    return pd.read_csv(fn, delim_whitespace=True, nrows=0).shape[1]

n_cols = get_header_length_csv(fn)

#   for run in runs:

if n_cols == 11:
    cols = ['time', 'L_gaze_X', 'L_gaze_Y', 'L_pupil_size', 'L_vel_X', 'L_vel_Y', 'null', 'target_x', 'target_y', 'target_distance', 'null2']
elif n_cols == 12:
    cols = ['time', 'L_gaze_X', 'L_gaze_Y', 'L_pupil_size', 'R_gaze_X', 'R_gaze_Y', 'R_pupil_size', 'L_vel_X', 'L_vel_Y', 'R_vel_X', 'R_vel_Y', 'null2']
elif n_cols == 7:
    cols = ['time', 'L_gaze_X', 'L_gaze_Y', 'L_pupil_size', 'L_vel_X', 'L_vel_Y', 'null']

e =pd.read_csv(fn,
                delim_whitespace=True, index_col=0,
                names=cols, na_values='.',
                usecols=['time', 'L_gaze_X', 'L_gaze_Y'])
e.fillna(method='ffill')
e.index = pd.TimedeltaIndex(e.index, unit='ms')
eyepos.append(e)

eyepos = pd.DataFrame(eyepos[0])


In [4]:

#median = eyepos['L_gaze_X'].median()
#bins = [-np.inf, median-350, median-100, median-75, median+75, median+100, median+350, np.inf]
median = 1000 # df['end_x'].median()
dist_to_median_out= 750
dist_to_median = 250
#bins = [-np.inf, median-350, median-100, median-75, median+75, median+100, median+350, np.inf]
bins = [-np.inf, median-dist_to_median_out, median-dist_to_median, median-75, median+75, median+dist_to_median, median+dist_to_median_out, np.inf]

eyepos['fixation_target']  = pd.cut(eyepos['L_gaze_X'], bins=bins,
                                    labels=['outside_left', 'left_option', 'center_left', 'fixation', 'center_right', 'right_option', 'outside_right'])

if trialwise:
    messages = subject.get_eyetracker_timings()
    messages = pd.to_timedelta(messages, unit='ms').unstack('type')

    # non-responses take 4500 ms
    messages.loc[messages['response'].isnull(), 'response'] = messages.loc[messages['response'].isnull()] + pd.to_timedelta(4500, 'ms')

    trialwise_eyepos = []
    for ix, row in messages.iterrows():
        if pd.notnull(row.stim) and pd.notnull(row.response):
            e = eyepos.loc[slice(row.stim, row.response)]
            trialwise_eyepos.append(e)
        else:
        # handle or skip this case
            continue
    trialwise_eyepos = pd.concat(trialwise_eyepos, keys=messages.index)

#return trialwise_eyepos


In [13]:
trialwise_eyepos = trialwise_eyepos.reset_index()
trialwise_eyepos['trial'] = trialwise_eyepos['trial'].astype(int)
trialwise_eyepos = trialwise_eyepos.set_index(['subject','trial','time'])

In [ ]:
eyepos = trialwise_eyepos.sort_index() # has 192 trials! 

# subject.get_trialwise_saccade_info(source='eyepos').
#eyepos = self.get_eyeposition(trialwise=True)

eyepos['n'] = eyepos.groupby(['trial'], group_keys=False)['fixation_target'].apply(lambda x: x.ne(x.shift(1)).cumsum())
saccades = eyepos.groupby(['trial', 'n', 'fixation_target']).apply(lambda d: d.iloc[-1].name[-1] - d.iloc[0].name[-1])
saccades = (saccades.dt.total_seconds() * 1000.).to_frame('duration').reset_index(['fixation_target', 'n'])
saccades = saccades[saccades.duration > 30.]

In [ ]:
saccades

,n,fixation_target,duration
trial,,,
0,1,center_left,530.0
0,2,left_option,388.0
0,3,center_left,174.0
0,6,fixation,118.0
0,8,fixation,76.0
...,...,...,...
192,41,center_left,122.0
192,42,left_option,232.0
192,46,right_option,766.0
